In [67]:
import sys
sys.path.append("../tools/")
import pdtb_utils
import pandas as pd
test_relations = pdtb_utils.get_train_relations()

In [68]:
df = pd.DataFrame([{'id': rel.relation_id(), 
                    'senses': tuple(rel.senses()), 
                    'relation_type': rel.relation_type(), 
                    'connective_token': rel.connective_token()} 
                   for rel in test_relations]).set_index('id')
df.describe()

,connective_token,relation_type,senses
count,32535,32535,32535
unique,754,4,88
top,,Explicit,"(Expansion.Conjunction,)"
freq,4133,14722,7355


In [69]:
df.groupby('relation_type').describe()

connective_token                    senses
relation_type                                                  
AltLex        count               524                       524
              unique              415                        18
              top      That's because      (Contingency.Cause,)
              freq                 11                       220
EntRel        count              4133                      4133
              unique                1                         1
              top                                     (EntRel,)
              freq               4133                      4133
Explicit      count             14722                     14722
              unique              324                        67
              top                 and  (Expansion.Conjunction,)
              freq               1811                      4126
Implicit      count             13156                     13156
              unique               95                        54
              top             because      (Contingency.Cause,)
              freq               1537                      3387

In [70]:
freq = pd.DataFrame(df.groupby('senses').count().sort_values('connective_token', ascending=False)['connective_token'])
freq['ratio'] = freq['connective_token'] / freq['connective_token'].sum()
with open('../paper/tables/sense_frequency.tex', 'w') as w:
    freq.head(20).to_latex(w)

## Initial thoughts

- Temporal contains typical temporal words, "years", "25 days later", etc. This should be fairly easy to solve by asking if it contains a certain set of temporal words.

## Most common discourse connective token

In [71]:
df.connective_token = df.connective_token.apply(str.lower)
df.senses = df.senses.apply(lambda x: "+".join(x))

In [72]:
token_rel_freq = df.groupby(['connective_token', 'senses']).count().sort_values('relation_type', ascending=False).rename(columns={'relation_type': 'freq'})
token_tot = df.groupby('connective_token').count()['senses']
token_ratio = token_rel_freq.divide(token_tot, axis=0, level=0).rename(columns={'freq': 'ratio'})
pd.concat([token_rel_freq, token_ratio], axis=1).sort_values('freq', ascending=False)

,,freq,ratio
connective_token,senses,,
,EntRel,4133,1.000000
and,Expansion.Conjunction,3192,0.931971
but,Comparison.Contrast,2290,0.702885
because,Contingency.Cause,2141,0.990745
also,Expansion.Conjunction,1793,0.997219
specifically,Expansion.Restatement,911,0.968119
so,Contingency.Cause,879,0.985426
if,Contingency.Condition,874,0.988688
for example,Expansion.Instantiation,744,0.917386
